# Objectives

-	Increase dpi and redo conversion CHECK 
-	Require a publisher (will avoid hand-drawn) CHECK
-	Top 50 composers for now in toy dataset
-	Elim orchestral scores
-	Make sure there aren’t cover pages, etc.
-	Make sure data in grayscale—standardize
-	Do as much of this as possible in an automated fashion. Publisher requirement will require some iteration and testing. 
-	ALSO re-do cell phone pictures, taking into account these constraints.


In [1]:
import glob
import os
import subprocess
import numpy as np
from scipy import ndimage
from scipy import signal
from scipy.misc import imsave
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.transform import rotate, resize
import time
import pylab as pl
from IPython import display
import librosa
import pickle
import pandas as pd
from bs4 import BeautifulSoup as bsoup
import shutil
from PyPDF2 import PdfFileWriter, PdfFileReader

## Convert w 300 dpi

In [ ]:
root_dir = 'score_data'
pdf_dir = root_dir + '/prepped_pdf' # directory containing pdfs of scores
png_dir = root_dir + '/prepped_png' # root output directory for png files

In [ ]:
#os.mkdir(png_dir)
pdf_files = glob(pdf_dir + '/*.pdf')
for pdf_file in pdf_files:
    basename = os.path.splitext(os.path.basename(pdf_file))[0]
    pngout = png_dir + '/' + basename + '.png'
    subprocess.call(['convert', '-density', '300', pdf_file, pngout]) # 72 dpi is the default value

## Rotate

In [ ]:
def importImage(pngfile):
    img = ndimage.imread(pngfile, flatten=True)
    img = 1 - img/255.0 # make white -> 0, black -> 1
    return img

In [ ]:
def showImage(X, sz = (12,12)):
    plt.figure(figsize=sz)
    plt.imshow(1-X, cmap='gray', vmin=0, vmax=1)

In [ ]:
def rotateToHorizontal(img, lb=-2, ub=2, incr=.01, topN=40):
    bestscore = -np.inf
    bestTheta = 0
    for theta in np.arange(lb, ub, incr):
        imgRot = rotate(img,theta)
        rsums = np.sum(imgRot, axis=1)
        rsums_sorted = sorted(rsums)[::-1]
        curscore = np.sum(rsums_sorted[0:topN])
        if curscore > bestscore:
            bestscore = curscore
            bestTheta = theta
    result = rotate(img,bestTheta)
    return result, bestTheta

In [ ]:
I = importImage('score_data/prepped_png/score17.png')
Irot, theta = rotateToHorizontal(I,-1,1)
showImage(Irot)

In [ ]:
def rotateAllImages(indir, outdir):
    png_files = glob.glob(indir + '/*.png')
    for png_file in png_files:
        I = importImage(png_file)
        Irot, theta = rotateToHorizontal(I)
        img = 255 - np.uint8(Irot*255)
        basename = os.path.basename(png_file)
        pngout = outdir + '/' + basename
        imsave(pngout,img)

In [ ]:
pngrot_dir = root_dir + '/toAnnotate' # rotated png files

In [ ]:
os.mkdir(pngrot_dir)
rotateAllImages(png_dir, pngrot_dir)

Here's an example of something w/o publisher: https://imslp.org/wiki/Acis_and_Galatea%2C_HWV_49_(Handel%2C_George_Frideric)
just go to the page parts and it's the first thing w/ id 106106

## Get bad (no publisher) names

We'll assume that we first loaded our text file as html text so we can parse with beautifulsoup. soup is the name of the object containing html info

In [ ]:
# open html text file as a beautifulsoup object
with open('html.txt') as f:
    soup = BeautifulSoup(f, "html.parser")

In [ ]:
for div in soup.find_all('div', class_ = 'we'):
    # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
    div = str(div)
    if 'Manuscript' in div or 'manuscript' in div:
        value_on = div[div.find('indexes='):] # this is where the ID should show up on a page
        bad_name = value_on[8:14] # just get the id, assuming 6 digits
        if bad_name[-1] = '/':
            # if it's a slash, there's something else after it
            bad_name_two = value_on[14:20]
            # check if last thing is actually a digit, in case of 5-digit ID
            if not bad_name_two[-1].isdigit():
                bad_name_two = bad_name_two[:-1]
            bad_pdf_two = bad_name_two + '.pdf'
            bad_names.append(bad_pdf_two)
        if not bad_name[-1].isdigit():
            # make could be a 5-digit ID
            bad_name = bad_name[:-1]
        bad_pdf = bad_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
        bad_names.append(bad_pdf) # append this to a list of bad names

## Check if Orchestral Score

Let's see if we can automate checking if smth is an orchestral score or just a part. Let's do a Beethoven Symphony :D

In [ ]:
divList = []
score_names = []

In [ ]:
for div in soup.find_all('div', class_ = 'we'):
    # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
    div = str(div)
    if 'Score' in div or 'score' in div:
        divList.append(div)

In [ ]:
for div in divList:
    value_on = div[div.find('indexes='):]
    score_name = value_on[8:14]
    if score_name[-1] = '/':
            # if it's a slash, there's something else after it
            score_name_two = value_on[14:20]
            # check if last thing is actually a digit, in case of 5-digit ID
            if not score_name_two[-1].isdigit():
                score_name_two = score_name_two[:-1]
            score_pdf_two = score_name_two + '.pdf'
            score_names.append(score_pdf_two)
    if not score_name[-1].isdigit():
        # make could be a 5-digit ID
        score_name = score_name[:-1]
    score_pdf = score_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
    score_names.append(score_pdf) # append this to a list of bad names

# Re-create Dataset

In [2]:
current_dir = '/data1/dbashir/Project/Summer2018/'
topFolder = 'pdf'
resultsFolder = 'results_top50'
intermedFolder = 'results_intermediate'
pdfMiniFolder = 'mini_dataset_pdf'
pngMiniFolder = 'mini_dataset_png'

dataSetSize = 70

resultsPath = os.path.join(topFolder,resultsFolder)
intermedPath = os.path.join(topFolder, intermedFolder)
intermedFromTop = os.path.join(current_dir, intermedPath)

pdfDir = os.path.join(current_dir, topFolder)
miniPDFDataset = os.path.join(topFolder, pdfMiniFolder)
miniPNGDataset = os.path.join(pdfDir, pngMiniFolder)
pngFromTop = os.path.join(topFolder, pngMiniFolder)
pdfFromTop = os.path.join(pdfDir, pdfMiniFolder)
rotPngFromTop = os.path.join(topFolder, 'mini_dataset_png_rot')

fileList = []

print(pdfDir)

/data1/dbashir/Project/Summer2018/pdf


In [3]:
if not os.path.exists(intermedPath):
    os.makedirs(intermedPath)

In [4]:
if not os.path.exists(miniPDFDataset):
    os.makedirs(miniPDFDataset)

In [5]:
if not os.path.exists(miniPNGDataset):
    os.makedirs(miniPNGDataset)

In [6]:
if not os.path.exists(rotPngFromTop):
    os.makedirs(rotPngFromTop)

In [3]:
def make_bad_pdfs(html_path):
    bad_names = []
    with open(html_path) as f:
        soup = bsoup(f, "html.parser")

    for div in soup.find_all('div', class_ = 'we'):
        # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
        trs = div.find_all('tr') # also get no publisher
        for tr in trs:
            tr = str(tr)
            try:
                if 'Pub' not in tr:
                    value_on = tr[tr.find('indexes='):]
                    bad_name = value_on[8:14]
                    if bad_name[-1] == '/':
                        # if it's a slash, there's something else after it
                        bad_name_two = value_on[14:20]
                        # check if last thing is actually a digit, in case of 5-digit ID
                        if not bad_name_two[-1].isdigit():
                            bad_name_two = bad_name_two[:-1]
                        bad_pdf_two = bad_name_two + '.pdf'
                        bad_names.append(bad_pdf_two)
                    if not bad_name[-1].isdigit():
                        # make could be a 5-digit ID
                        bad_name = bad_name[:-1]
                    bad_pdf = bad_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
                    bad_names.append(bad_pdf) # append this to a list of bad name
            except:
                continue
        div = str(div)
        try:
            if 'Manuscript' in div or 'manuscript' in div:
                value_on = div[div.find('indexes='):] # this is where the ID should show up on a page
                bad_name_loc = value_on[8:] # just get the id, assuming 6 digits
                location_letter = 0 # so we can  go from beginning of bad_name_loc to the first letter and get all IDs
                while not bad_name_loc[location_letter].isalpha():
                    location_letter += 1
                # now check for a quote or smth
                names_location = bad_name_loc[0:location_letter]
                # get last number, going from the back
                last_num = next(i for i,j in list(enumerate(names_location,1))[::-1] if j.isdigit())
                if not names_location[-1].isnumeric():
                    names_location = names_location[:-1]
                names_location = names_location[:last_num]
                # now split by slashes to get the numbers
                IDs = names_location.split('/')
                IDs = [ID + '.pdf' for ID in IDs]
                for bad_name in IDs:
                    bad_names.append(bad_name)
                
                # now find and remove slashes
#                 if bad_name[-1] == '/':
#                     # if it's a slash, there's something else after it
#                     bad_name_two = value_on[14:20]
#                     # check if last thing is actually a digit, in case of 5-digit ID
#                     if not bad_name_two[-1].isdigit():
#                         bad_name_two = bad_name_two[:-1]
#                     bad_pdf_two = bad_name_two + '.pdf'
#                     bad_names.append(bad_pdf_two)
#                 if not bad_name[-1].isdigit():
#                     # make could be a 5-digit ID
#                     bad_name = bad_name[:-1]
                #bad_pdf = bad_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
                #bad_names.append(bad_pdf) # append this to a list of bad names
        except:
            continue
            
    return bad_names

In [7]:
def make_score_pdfs(html_path):
    score_names = []
    divList = []
    with open(html_path) as f:
        soup = bsoup(f, "html.parser")
    for div in soup.find_all('div', class_ = 'we'):
        # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
        div = str(div)
        if 'Score' in div or 'score' in div:
            divList.append(div)
    for div in divList:
        try:
            value_on = div[div.find('indexes='):] # this is where the ID should show up on a page
            bad_name_loc = value_on[8:] # just get the id, assuming 6 digits
            location_letter = 0 # so we can  go from beginning of bad_name_loc to the first letter and get all IDs
            while not bad_name_loc[location_letter].isalpha():
                location_letter += 1
            # now check for a quote or smth
            names_location = bad_name_loc[0:location_letter]
            # get last number, going from the back
            last_num = next(i for i,j in list(enumerate(names_location,1))[::-1] if j.isdigit())
            if not names_location[-1].isnumeric():
                names_location = names_location[:-1]
            names_location = names_location[:last_num]
            # now split by slashes to get the numbers
            IDs = names_location.split('/')
            IDs = [ID + '.pdf' for ID in IDs]
            for bad_name in IDs:
                bad_names.append(bad_name)
        except:
            continue
    return score_names

In [7]:
#using os.walk, move all the .pdf files to an intermediate folder for EZ access
all_bad_names = []
all_score_names = []
for subdir, dirs, files in os.walk(resultsPath):
    for pdf_file in files:
        if 'pdf' in pdf_file:
            #move it to intermediate file for shuffling
#             print(pdf_file)
#             print("subdirectory: " + subdir)
            html_path = os.path.join(subdir, 'html.txt')
            bad_names = make_bad_pdfs(html_path)
            score_names = make_score_pdfs(html_path)
            for bad_name in bad_names:
                all_bad_names.append(bad_name)
            for score_name in score_names:
                all_score_names.append(score_name)
            if pdf_file in bad_names or pdf_file in score_names:
                print("skipping file: " + str(pdf_file))
                continue # skip if it's a handwritten or orchestral score
            if pdf_file not in all_bad_names or pdf_file not in all_score_names:
                pdf_file_path = os.path.join(subdir, pdf_file)
                final_path = os.path.join(intermedPath, pdf_file)
                shutil.copy(pdf_file_path, final_path)

skipping file: 270629.pdf
skipping file: 159741.pdf
skipping file: 42540.pdf
skipping file: 20515.pdf
skipping file: 87055.pdf
skipping file: 270796.pdf
skipping file: 234591.pdf
skipping file: 143296.pdf
skipping file: 272417.pdf
skipping file: 58813.pdf
skipping file: 68459.pdf
skipping file: 270430.pdf
skipping file: 105269.pdf
skipping file: 452822.pdf
skipping file: 42541.pdf
skipping file: 15191.pdf
skipping file: 270632.pdf
skipping file: 385573.pdf
skipping file: 411838.pdf
skipping file: 87056.pdf
skipping file: 240429.pdf
skipping file: 16144.pdf
skipping file: 14555.pdf
skipping file: 504187.pdf
skipping file: 270423.pdf
skipping file: 322115.pdf
skipping file: 385686.pdf
skipping file: 385730.pdf
skipping file: 385691.pdf
skipping file: 337078.pdf
skipping file: 180677.pdf
skipping file: 385711.pdf
skipping file: 383664.pdf
skipping file: 520316.pdf
skipping file: 505600.pdf
skipping file: 291632.pdf
skipping file: 291628.pdf
skipping file: 420951.pdf
skipping file: 102874.

skipping file: 385684.pdf
skipping file: 291635.pdf
skipping file: 420949.pdf
skipping file: 162119.pdf
skipping file: 162084.pdf
skipping file: 117300.pdf
skipping file: 291630.pdf
skipping file: 413960.pdf
skipping file: 57966.pdf
skipping file: 453649.pdf
skipping file: 93976.pdf
skipping file: 93975.pdf
skipping file: 130233.pdf
skipping file: 505630.pdf
skipping file: 184632.pdf
skipping file: 163509.pdf
skipping file: 234233.pdf
skipping file: 520314.pdf
skipping file: 21738.pdf
skipping file: 288708.pdf
skipping file: 228156.pdf
skipping file: 105449.pdf
skipping file: 244394.pdf
skipping file: 21739.pdf
skipping file: 445410.pdf
skipping file: 59243.pdf
skipping file: 24725.pdf
skipping file: 389006.pdf
skipping file: 322157.pdf
skipping file: 381909.pdf
skipping file: 147291.pdf
skipping file: 513937.pdf
skipping file: 385569.pdf
skipping file: 31791.pdf
skipping file: 115015.pdf
skipping file: 111897.pdf
skipping file: 383663.pdf
skipping file: 109840.pdf
skipping file: 32565

skipping file: 423923.pdf
skipping file: 139560.pdf
skipping file: 214305.pdf
skipping file: 114797.pdf
skipping file: 13304.pdf
skipping file: 73176.pdf
skipping file: 84319.pdf
skipping file: 58603.pdf
skipping file: 109379.pdf
skipping file: 338494.pdf
skipping file: 326669.pdf
skipping file: 131135.pdf
skipping file: 253352.pdf
skipping file: 253345.pdf
skipping file: 200901.pdf
skipping file: 253343.pdf
skipping file: 253349.pdf
skipping file: 115659.pdf
skipping file: 476776.pdf
skipping file: 464518.pdf
skipping file: 404683.pdf
skipping file: 445422.pdf
skipping file: 111609.pdf
skipping file: 404685.pdf
skipping file: 73181.pdf
skipping file: 58604.pdf
skipping file: 116266.pdf
skipping file: 475247.pdf
skipping file: 253353.pdf
skipping file: 430524.pdf
skipping file: 113041.pdf
skipping file: 409318.pdf
skipping file: 90538.pdf
skipping file: 112902.pdf
skipping file: 58606.pdf
skipping file: 377338.pdf
skipping file: 140524.pdf
skipping file: 253318.pdf
skipping file: 11290

skipping file: 323197.pdf
skipping file: 334118.pdf
skipping file: 334112.pdf
skipping file: 252876.pdf
skipping file: 323186.pdf
skipping file: 295105.pdf
skipping file: 16776.pdf
skipping file: 323169.pdf
skipping file: 186524.pdf
skipping file: 356213.pdf
skipping file: 260796.pdf
skipping file: 75869.pdf
skipping file: 496364.pdf
skipping file: 350486.pdf
skipping file: 356251.pdf
skipping file: 444472.pdf
skipping file: 253257.pdf
skipping file: 250777.pdf
skipping file: 247231.pdf
skipping file: 250781.pdf
skipping file: 356219.pdf
skipping file: 247193.pdf
skipping file: 323293.pdf
skipping file: 380212.pdf
skipping file: 383182.pdf
skipping file: 311220.pdf
skipping file: 250501.pdf
skipping file: 261384.pdf
skipping file: 114339.pdf
skipping file: 323176.pdf
skipping file: 323247.pdf
skipping file: 257072.pdf
skipping file: 252007.pdf
skipping file: 451336.pdf
skipping file: 306207.pdf
skipping file: 299453.pdf
skipping file: 363028.pdf
skipping file: 255106.pdf
skipping file:

skipping file: 261371.pdf
skipping file: 250526.pdf
skipping file: 250955.pdf
skipping file: 250679.pdf
skipping file: 250794.pdf
skipping file: 114355.pdf
skipping file: 323242.pdf
skipping file: 16789.pdf
skipping file: 250941.pdf
skipping file: 255110.pdf
skipping file: 356196.pdf
skipping file: 253255.pdf
skipping file: 260794.pdf
skipping file: 356198.pdf
skipping file: 252019.pdf
skipping file: 251839.pdf
skipping file: 295045.pdf
skipping file: 418523.pdf
skipping file: 261377.pdf
skipping file: 380210.pdf
skipping file: 323288.pdf
skipping file: 356246.pdf
skipping file: 166243.pdf
skipping file: 356267.pdf
skipping file: 250954.pdf
skipping file: 247160.pdf
skipping file: 261389.pdf
skipping file: 418515.pdf
skipping file: 312735.pdf
skipping file: 323178.pdf
skipping file: 247233.pdf
skipping file: 16799.pdf
skipping file: 356069.pdf
skipping file: 363035.pdf
skipping file: 356195.pdf
skipping file: 323239.pdf
skipping file: 16786.pdf
skipping file: 250795.pdf
skipping file: 

...

skipping file: 360740.pdf
skipping file: 32856.pdf
skipping file: 295033.pdf
skipping file: 266255.pdf
skipping file: 295062.pdf
skipping file: 32857.pdf
skipping file: 407469.pdf
skipping file: 295032.pdf
skipping file: 338270.pdf
skipping file: 368622.pdf
skipping file: 341176.pdf
skipping file: 331384.pdf
skipping file: 433849.pdf
skipping file: 386425.pdf
skipping file: 333317.pdf
skipping file: 335235.pdf
skipping file: 54374.pdf
skipping file: 326558.pdf
skipping file: 372162.pdf
skipping file: 343423.pdf
skipping file: 429087.pdf
skipping file: 380486.pdf
skipping file: 153471.pdf
skipping file: 54269.pdf
skipping file: 337118.pdf
skipping file: 357125.pdf
skipping file: 266103.pdf
skipping file: 388172.pdf
skipping file: 263105.pdf
skipping file: 332966.pdf
skipping file: 112576.pdf
skipping file: 340058.pdf
skipping file: 306559.pdf
skipping file: 338050.pdf
skipping file: 495227.pdf
skipping file: 338326.pdf
skipping file: 395350.pdf
skipping file: 495506.pdf
skipping file: 1

skipping file: 112430.pdf
skipping file: 203385.pdf
skipping file: 22824.pdf
skipping file: 107618.pdf
skipping file: 250277.pdf
skipping file: 138164.pdf
skipping file: 102705.pdf
skipping file: 108883.pdf
skipping file: 311060.pdf
skipping file: 108880.pdf
skipping file: 333873.pdf
skipping file: 317582.pdf
skipping file: 17180.pdf
skipping file: 345626.pdf
skipping file: 493377.pdf
skipping file: 22852.pdf
skipping file: 147277.pdf
skipping file: 516805.pdf
skipping file: 22893.pdf
skipping file: 138586.pdf
skipping file: 107694.pdf
skipping file: 374716.pdf
skipping file: 351208.pdf
skipping file: 107642.pdf
skipping file: 103883.pdf
skipping file: 120918.pdf
skipping file: 34589.pdf
skipping file: 23170.pdf
skipping file: 438454.pdf
skipping file: 471426.pdf
skipping file: 22845.pdf
skipping file: 110011.pdf
skipping file: 10985.pdf
skipping file: 109042.pdf
skipping file: 110010.pdf
skipping file: 97823.pdf
skipping file: 22891.pdf
skipping file: 97822.pdf
skipping file: 367180.p

skipping file: 84162.pdf
skipping file: 516786.pdf
skipping file: 22825.pdf
skipping file: 302004.pdf
skipping file: 352023.pdf
skipping file: 246833.pdf
skipping file: 41943.pdf
skipping file: 77174.pdf
skipping file: 41942.pdf
skipping file: 77173.pdf
skipping file: 23099.pdf
skipping file: 246134.pdf
skipping file: 22901.pdf
skipping file: 101428.pdf
skipping file: 106925.pdf
skipping file: 13154.pdf
skipping file: 22986.pdf
skipping file: 177612.pdf
skipping file: 154584.pdf
skipping file: 23199.pdf
skipping file: 494811.pdf
skipping file: 88081.pdf
skipping file: 88080.pdf
skipping file: 41549.pdf
skipping file: 309180.pdf
skipping file: 81993.pdf
skipping file: 23078.pdf
skipping file: 81994.pdf
skipping file: 41548.pdf
skipping file: 174153.pdf
skipping file: 22821.pdf
skipping file: 97689.pdf
skipping file: 82756.pdf
skipping file: 97690.pdf
skipping file: 82757.pdf
skipping file: 245548.pdf
skipping file: 13512.pdf
skipping file: 23198.pdf
skipping file: 347105.pdf
skipping fi

skipping file: 334901.pdf
skipping file: 117525.pdf
skipping file: 317803.pdf
skipping file: 12057.pdf
skipping file: 475809.pdf
skipping file: 516790.pdf
skipping file: 109219.pdf
skipping file: 34540.pdf
skipping file: 102641.pdf
skipping file: 167816.pdf
skipping file: 23081.pdf
skipping file: 108868.pdf
skipping file: 88084.pdf
skipping file: 88085.pdf
skipping file: 13158.pdf
skipping file: 507506.pdf
skipping file: 163728.pdf
skipping file: 12062.pdf
skipping file: 317596.pdf
skipping file: 466339.pdf
skipping file: 34543.pdf
skipping file: 411857.pdf
skipping file: 23141.pdf
skipping file: 23080.pdf
skipping file: 97829.pdf
skipping file: 97830.pdf
skipping file: 71097.pdf
skipping file: 18356.pdf
skipping file: 71096.pdf
skipping file: 71153.pdf
skipping file: 22992.pdf
skipping file: 517080.pdf
skipping file: 30065.pdf
skipping file: 56206.pdf
skipping file: 112439.pdf
skipping file: 84396.pdf
skipping file: 22810.pdf
skipping file: 208169.pdf
skipping file: 107614.pdf
skippin

skipping file: 22137.pdf
skipping file: 52969.pdf
skipping file: 240160.pdf
skipping file: 58926.pdf
skipping file: 521515.pdf
skipping file: 40234.pdf
skipping file: 52706.pdf
skipping file: 173180.pdf
skipping file: 47234.pdf
skipping file: 78069.pdf
skipping file: 458349.pdf
skipping file: 155103.pdf
skipping file: 78070.pdf
skipping file: 343916.pdf
skipping file: 262342.pdf
skipping file: 36912.pdf
skipping file: 36913.pdf
skipping file: 71415.pdf
skipping file: 11635.pdf
skipping file: 37445.pdf
skipping file: 170290.pdf
skipping file: 333019.pdf
skipping file: 423241.pdf
skipping file: 175902.pdf
skipping file: 11658.pdf
skipping file: 96276.pdf
skipping file: 316544.pdf
skipping file: 33523.pdf
skipping file: 59514.pdf
skipping file: 232149.pdf
skipping file: 58239.pdf
skipping file: 229739.pdf
skipping file: 23355.pdf
skipping file: 291379.pdf
skipping file: 232979.pdf
skipping file: 29965.pdf
skipping file: 12262.pdf
skipping file: 56667.pdf
skipping file: 103487.pdf
skipping

In [ ]:
# remove bad files from intermed folder:
#using os.walk, move all the .pdf files to an intermediate folder for EZ access
all_bad_names = []
all_score_names = []
for subdir, dirs, files in os.walk(resultsPath):
    for pdf_file in files:
        if 'pdf' in pdf_file:
            html_path = os.path.join(subdir, 'html.txt')
            bad_names = make_bad_pdfs(html_path)
            score_names = make_score_pdfs(html_path)
            for bad_name in bad_names:
                all_bad_names.append(bad_name)
            for score_name in score_names:
                all_score_names.append(score_name)
            if pdf_file in bad_names or pdf_file in score_names:
                if pdf_file in os.listdir(intermedPath):
                    print("removing file: " + str(pdf_file))
                    os.remove(os.path.join(intermedPath, pdf_file))
                else:
                    continue
            if pdf_file in all_bad_names or pdf_file in all_score_names:
                if pdf_file in os.listdir(intermedPath):
                    rmv_path = os.path.join(intermedPath, pdf_file)
                    print("about to remove: " + str(rmv_path))
                    os.remove(rmv_path)
                else:
                    continue

removing file: 272418.pdf
removing file: 143297.pdf
removing file: 240430.pdf
removing file: 77568.pdf
removing file: 385712.pdf
removing file: 180678.pdf
removing file: 450215.pdf
removing file: 450401.pdf
removing file: 450407.pdf
removing file: 450402.pdf
removing file: 450404.pdf
removing file: 450408.pdf
removing file: 450216.pdf
removing file: 450405.pdf
removing file: 450409.pdf
removing file: 450403.pdf
removing file: 453848.pdf
removing file: 378201.pdf
removing file: 236847.pdf
removing file: 225689.pdf
removing file: 77563.pdf
removing file: 50975.pdf
removing file: 504189.pdf
removing file: 382034.pdf
removing file: 111894.pdf
removing file: 105452.pdf
removing file: 18578.pdf
removing file: 111620.pdf
removing file: 423983.pdf
removing file: 377820.pdf
removing file: 77564.pdf
removing file: 421428.pdf
removing file: 290401.pdf
removing file: 290400.pdf
removing file: 225449.pdf
removing file: 225450.pdf
removing file: 512603.pdf
removing file: 110881.pdf
removing file: 30

removing file: 190122.pdf
removing file: 190117.pdf
removing file: 190143.pdf
removing file: 383422.pdf
removing file: 418533.pdf
removing file: 383424.pdf
removing file: 114356.pdf
removing file: 141078.pdf
removing file: 363036.pdf
removing file: 114358.pdf
removing file: 408713.pdf
removing file: 348445.pdf
removing file: 348444.pdf
removing file: 348453.pdf
removing file: 348455.pdf
removing file: 348457.pdf
removing file: 439265.pdf
removing file: 348443.pdf
removing file: 16169.pdf
removing file: 348447.pdf
removing file: 348442.pdf
removing file: 348460.pdf
removing file: 348446.pdf
removing file: 294852.pdf
removing file: 348448.pdf
removing file: 95931.pdf
removing file: 348462.pdf
removing file: 348461.pdf
removing file: 294854.pdf
removing file: 348454.pdf
removing file: 348456.pdf
removing file: 348459.pdf
removing file: 348458.pdf
removing file: 348449.pdf
removing file: 348440.pdf
removing file: 348451.pdf
removing file: 348441.pdf
removing file: 348463.pdf
removing file:

removing file: 449193.pdf
removing file: 61851.pdf
removing file: 61853.pdf
removing file: 61850.pdf
removing file: 108611.pdf
removing file: 112353.pdf
removing file: 28872.pdf
removing file: 61527.pdf
removing file: 332353.pdf
removing file: 484504.pdf
removing file: 108610.pdf
removing file: 484503.pdf
removing file: 484501.pdf
removing file: 61852.pdf
removing file: 108614.pdf
removing file: 66417.pdf
removing file: 205176.pdf
removing file: 400437.pdf
removing file: 184884.pdf
removing file: 117593.pdf
removing file: 74135.pdf
removing file: 367701.pdf
removing file: 367702.pdf
removing file: 367703.pdf
removing file: 400442.pdf
removing file: 252896.pdf
removing file: 53188.pdf
removing file: 28596.pdf
removing file: 251996.pdf
removing file: 166822.pdf
removing file: 251995.pdf
removing file: 166819.pdf
removing file: 166820.pdf
removing file: 251992.pdf
removing file: 166821.pdf
removing file: 28597.pdf
removing file: 251994.pdf
removing file: 251997.pdf
removing file: 47674.pd

removing file: 466507.pdf
removing file: 318878.pdf
removing file: 310222.pdf
removing file: 445621.pdf
removing file: 343962.pdf
removing file: 453426.pdf
removing file: 332834.pdf
removing file: 466504.pdf
removing file: 392349.pdf
removing file: 349850.pdf
removing file: 349852.pdf
removing file: 392343.pdf
removing file: 392348.pdf
removing file: 392350.pdf
removing file: 392346.pdf
removing file: 349854.pdf
removing file: 349855.pdf
removing file: 349856.pdf
removing file: 392345.pdf
removing file: 349849.pdf
removing file: 349853.pdf
removing file: 392347.pdf
removing file: 392344.pdf
removing file: 349851.pdf
removing file: 450098.pdf
removing file: 450099.pdf
removing file: 334626.pdf
removing file: 334627.pdf
removing file: 338912.pdf
removing file: 335797.pdf
removing file: 350152.pdf
removing file: 342019.pdf
removing file: 338913.pdf
removing file: 311068.pdf
removing file: 319395.pdf
removing file: 319396.pdf
removing file: 319397.pdf
removing file: 311086.pdf
removing fil

removing file: 499111.pdf
removing file: 135560.pdf
removing file: 290941.pdf
removing file: 486857.pdf
removing file: 486864.pdf
removing file: 486865.pdf
removing file: 486861.pdf
removing file: 486859.pdf
removing file: 486866.pdf
removing file: 486863.pdf
removing file: 297947.pdf
removing file: 297948.pdf
removing file: 297946.pdf
removing file: 493927.pdf
removing file: 428078.pdf
removing file: 393038.pdf
removing file: 428077.pdf
removing file: 248393.pdf
removing file: 297758.pdf
removing file: 334790.pdf
removing file: 376713.pdf
removing file: 376714.pdf
removing file: 376715.pdf
removing file: 297759.pdf
removing file: 297760.pdf
removing file: 211195.pdf
removing file: 350073.pdf
removing file: 436198.pdf
removing file: 350076.pdf
removing file: 350077.pdf
removing file: 214466.pdf
removing file: 312600.pdf
removing file: 133745.pdf
removing file: 350072.pdf
removing file: 214465.pdf
removing file: 350071.pdf
removing file: 350075.pdf
removing file: 312599.pdf
removing fil

removing file: 362270.pdf
removing file: 464812.pdf
removing file: 297882.pdf
removing file: 358127.pdf
removing file: 362273.pdf
removing file: 16314.pdf
removing file: 272348.pdf
removing file: 135569.pdf
removing file: 500258.pdf
removing file: 210727.pdf
removing file: 210726.pdf
removing file: 502498.pdf
removing file: 482635.pdf
removing file: 502499.pdf
removing file: 134643.pdf
removing file: 502497.pdf
removing file: 225760.pdf
removing file: 225759.pdf
removing file: 225761.pdf
removing file: 460803.pdf
removing file: 487064.pdf
removing file: 485136.pdf
removing file: 485138.pdf
removing file: 485137.pdf
removing file: 483186.pdf
removing file: 131284.pdf
removing file: 134669.pdf
removing file: 240137.pdf
removing file: 240138.pdf
removing file: 500276.pdf
removing file: 240139.pdf
removing file: 309072.pdf
removing file: 309075.pdf
removing file: 309074.pdf
removing file: 309073.pdf
removing file: 436851.pdf
removing file: 494760.pdf
removing file: 426627.pdf
removing file

removing file: 250991.pdf
removing file: 168836.pdf
removing file: 221121.pdf
removing file: 221122.pdf
removing file: 135181.pdf
removing file: 495480.pdf
removing file: 432848.pdf
removing file: 260352.pdf
removing file: 260354.pdf
removing file: 248919.pdf
removing file: 260353.pdf
removing file: 279535.pdf
removing file: 485153.pdf
removing file: 485152.pdf
removing file: 161942.pdf
removing file: 502594.pdf
removing file: 161943.pdf
removing file: 122249.pdf
removing file: 129212.pdf
removing file: 162049.pdf
removing file: 502593.pdf
removing file: 338602.pdf
removing file: 429227.pdf
removing file: 502596.pdf
removing file: 504804.pdf
removing file: 502598.pdf
removing file: 122043.pdf
removing file: 338651.pdf
removing file: 338603.pdf
removing file: 502597.pdf
removing file: 122247.pdf
removing file: 122248.pdf
removing file: 338601.pdf
removing file: 338604.pdf
removing file: 429228.pdf
removing file: 161941.pdf
removing file: 161944.pdf
removing file: 128960.pdf
removing fil

In [34]:
%cd ttemp/Project/Summer2018/

/data1/dbashir/Project/Summer2018


In [36]:
print(miniPDFDataset)

pdf/mini_dataset_pdf


In [8]:
extra_dataset = 'pdf/extras_to_check'

In [16]:
#change directory to the intermediate folder
# if os.getcwd() != intermedFromTop:
#     os.chdir(intermedPath)
g = glob.glob(os.path.join(intermedPath,'*.pdf'))
shuf = np.random.permutation(g)
for i in range(1):
    name = shuf[i]
    name = name[25:]
    #os.rename(shuf[i], os.path.join(extra_dataset, name)) # also add so we can look at just these 5
    os.rename(shuf[i], os.path.join(miniPDFDataset, name))

Before splitting, I'll examine this small dataset manually to see how it looks.

In [11]:
def pdf_splitter(path):
    fname = os.path.splitext(os.path.basename(path))[0]
    
    #print(path)
    path = os.path.join(miniPDFDataset, path)
 
    pdf = PdfFileReader(open(path,"rb"))
    for page in range(pdf.getNumPages()):
        pdf_writer = PdfFileWriter()
        pdf_writer.addPage(pdf.getPage(page))
 
        output_filename = '{}_page_{}.pdf'.format(
            fname, page+1)
    
        output_filename = os.path.join(miniPDFDataset, output_filename)
 
        with open(output_filename, 'wb') as out:
            pdf_writer.write(out)
 
        print('Created: {}'.format(output_filename))

In [ ]:
os.chdir(pdfFromTop)
os.getcwd()

In [13]:
for pdf_file in os.listdir(miniPDFDataset):
    try:
        pdf_splitter(pdf_file)
    except:
        continue

Created: pdf/mini_dataset_pdf/113063_page_1.pdf
Created: pdf/mini_dataset_pdf/113063_page_2.pdf
Created: pdf/mini_dataset_pdf/113063_page_3.pdf
Created: pdf/mini_dataset_pdf/113063_page_4.pdf
Created: pdf/mini_dataset_pdf/113063_page_5.pdf
Created: pdf/mini_dataset_pdf/113063_page_6.pdf
Created: pdf/mini_dataset_pdf/113063_page_7.pdf
Created: pdf/mini_dataset_pdf/113063_page_8.pdf
Created: pdf/mini_dataset_pdf/113063_page_9.pdf
Created: pdf/mini_dataset_pdf/113063_page_10.pdf
Created: pdf/mini_dataset_pdf/113063_page_11.pdf
Created: pdf/mini_dataset_pdf/113063_page_12.pdf
Created: pdf/mini_dataset_pdf/113063_page_13.pdf
Created: pdf/mini_dataset_pdf/113063_page_14.pdf
Created: pdf/mini_dataset_pdf/113063_page_15.pdf
Created: pdf/mini_dataset_pdf/113063_page_16.pdf
Created: pdf/mini_dataset_pdf/113063_page_17.pdf
Created: pdf/mini_dataset_pdf/113063_page_18.pdf
Created: pdf/mini_dataset_pdf/113063_page_19.pdf
Created: pdf/mini_dataset_pdf/113063_page_20.pdf
Created: pdf/mini_dataset_pdf

Created: pdf/mini_dataset_pdf/236213_page_5.pdf
Created: pdf/mini_dataset_pdf/236213_page_6.pdf
Created: pdf/mini_dataset_pdf/236213_page_7.pdf
Created: pdf/mini_dataset_pdf/236213_page_8.pdf
Created: pdf/mini_dataset_pdf/236213_page_9.pdf
Created: pdf/mini_dataset_pdf/236213_page_10.pdf
Created: pdf/mini_dataset_pdf/236213_page_11.pdf
Created: pdf/mini_dataset_pdf/236213_page_12.pdf
Created: pdf/mini_dataset_pdf/27233_page_1.pdf
Created: pdf/mini_dataset_pdf/27233_page_2.pdf
Created: pdf/mini_dataset_pdf/27233_page_3.pdf
Created: pdf/mini_dataset_pdf/27233_page_4.pdf
Created: pdf/mini_dataset_pdf/27233_page_5.pdf
Created: pdf/mini_dataset_pdf/441592_page_1.pdf
Created: pdf/mini_dataset_pdf/441592_page_2.pdf
Created: pdf/mini_dataset_pdf/246294_page_1.pdf
Created: pdf/mini_dataset_pdf/246294_page_2.pdf
Created: pdf/mini_dataset_pdf/246294_page_3.pdf
Created: pdf/mini_dataset_pdf/246294_page_4.pdf
Created: pdf/mini_dataset_pdf/246294_page_5.pdf
Created: pdf/mini_dataset_pdf/03287_page_1

Created: pdf/mini_dataset_pdf/407960_page_1.pdf
Created: pdf/mini_dataset_pdf/407960_page_2.pdf
Created: pdf/mini_dataset_pdf/407960_page_3.pdf
Created: pdf/mini_dataset_pdf/407960_page_4.pdf
Created: pdf/mini_dataset_pdf/407960_page_5.pdf
Created: pdf/mini_dataset_pdf/407960_page_6.pdf
Created: pdf/mini_dataset_pdf/407960_page_7.pdf
Created: pdf/mini_dataset_pdf/319376_page_1.pdf
Created: pdf/mini_dataset_pdf/319376_page_2.pdf
Created: pdf/mini_dataset_pdf/319376_page_3.pdf
Created: pdf/mini_dataset_pdf/319376_page_4.pdf
Created: pdf/mini_dataset_pdf/167982_page_1.pdf
Created: pdf/mini_dataset_pdf/167982_page_2.pdf
Created: pdf/mini_dataset_pdf/08796_page_1.pdf
Created: pdf/mini_dataset_pdf/08796_page_2.pdf
Created: pdf/mini_dataset_pdf/08796_page_3.pdf
Created: pdf/mini_dataset_pdf/49635_page_1.pdf
Created: pdf/mini_dataset_pdf/49635_page_2.pdf
Created: pdf/mini_dataset_pdf/49635_page_3.pdf
Created: pdf/mini_dataset_pdf/49635_page_4.pdf
Created: pdf/mini_dataset_pdf/49635_page_5.pdf


Created: pdf/mini_dataset_pdf/53481_page_1.pdf
Created: pdf/mini_dataset_pdf/53481_page_2.pdf
Created: pdf/mini_dataset_pdf/53481_page_3.pdf
Created: pdf/mini_dataset_pdf/53481_page_4.pdf
Created: pdf/mini_dataset_pdf/53481_page_5.pdf
Created: pdf/mini_dataset_pdf/53481_page_6.pdf
Created: pdf/mini_dataset_pdf/53481_page_7.pdf
Created: pdf/mini_dataset_pdf/53481_page_8.pdf
Created: pdf/mini_dataset_pdf/53481_page_9.pdf
Created: pdf/mini_dataset_pdf/53481_page_10.pdf
Created: pdf/mini_dataset_pdf/53481_page_11.pdf
Created: pdf/mini_dataset_pdf/53481_page_12.pdf
Created: pdf/mini_dataset_pdf/53481_page_13.pdf
Created: pdf/mini_dataset_pdf/111403_page_1.pdf
Created: pdf/mini_dataset_pdf/111403_page_2.pdf
Created: pdf/mini_dataset_pdf/53381_page_1.pdf
Created: pdf/mini_dataset_pdf/53381_page_2.pdf
Created: pdf/mini_dataset_pdf/53381_page_3.pdf
Created: pdf/mini_dataset_pdf/53381_page_4.pdf
Created: pdf/mini_dataset_pdf/53381_page_5.pdf
Created: pdf/mini_dataset_pdf/377645_page_1.pdf
Create

Created: pdf/mini_dataset_pdf/105172_page_1.pdf
Created: pdf/mini_dataset_pdf/105172_page_2.pdf
Created: pdf/mini_dataset_pdf/105172_page_3.pdf
Created: pdf/mini_dataset_pdf/105172_page_4.pdf
Created: pdf/mini_dataset_pdf/105172_page_5.pdf
Created: pdf/mini_dataset_pdf/105172_page_6.pdf
Created: pdf/mini_dataset_pdf/105172_page_7.pdf
Created: pdf/mini_dataset_pdf/105172_page_8.pdf
Created: pdf/mini_dataset_pdf/105172_page_9.pdf
Created: pdf/mini_dataset_pdf/105172_page_10.pdf
Created: pdf/mini_dataset_pdf/105172_page_11.pdf
Created: pdf/mini_dataset_pdf/105172_page_12.pdf
Created: pdf/mini_dataset_pdf/105172_page_13.pdf
Created: pdf/mini_dataset_pdf/105172_page_14.pdf
Created: pdf/mini_dataset_pdf/105172_page_15.pdf
Created: pdf/mini_dataset_pdf/105172_page_16.pdf
Created: pdf/mini_dataset_pdf/105172_page_17.pdf
Created: pdf/mini_dataset_pdf/105172_page_18.pdf
Created: pdf/mini_dataset_pdf/105172_page_19.pdf
Created: pdf/mini_dataset_pdf/105172_page_20.pdf
Created: pdf/mini_dataset_pdf

Created: pdf/mini_dataset_pdf/105172_page_179.pdf
Created: pdf/mini_dataset_pdf/105172_page_180.pdf
Created: pdf/mini_dataset_pdf/105172_page_181.pdf
Created: pdf/mini_dataset_pdf/105172_page_182.pdf
Created: pdf/mini_dataset_pdf/105172_page_183.pdf
Created: pdf/mini_dataset_pdf/105172_page_184.pdf
Created: pdf/mini_dataset_pdf/105172_page_185.pdf
Created: pdf/mini_dataset_pdf/105172_page_186.pdf
Created: pdf/mini_dataset_pdf/105172_page_187.pdf
Created: pdf/mini_dataset_pdf/105172_page_188.pdf
Created: pdf/mini_dataset_pdf/105172_page_189.pdf
Created: pdf/mini_dataset_pdf/105172_page_190.pdf
Created: pdf/mini_dataset_pdf/105172_page_191.pdf
Created: pdf/mini_dataset_pdf/105172_page_192.pdf
Created: pdf/mini_dataset_pdf/105172_page_193.pdf
Created: pdf/mini_dataset_pdf/105172_page_194.pdf
Created: pdf/mini_dataset_pdf/105172_page_195.pdf
Created: pdf/mini_dataset_pdf/105172_page_196.pdf
Created: pdf/mini_dataset_pdf/105172_page_197.pdf
Created: pdf/mini_dataset_pdf/105172_page_198.pdf


Created: pdf/mini_dataset_pdf/351735_page_21.pdf
Created: pdf/mini_dataset_pdf/351735_page_22.pdf
Created: pdf/mini_dataset_pdf/351735_page_23.pdf
Created: pdf/mini_dataset_pdf/351735_page_24.pdf
Created: pdf/mini_dataset_pdf/351735_page_25.pdf
Created: pdf/mini_dataset_pdf/351735_page_26.pdf
Created: pdf/mini_dataset_pdf/351735_page_27.pdf
Created: pdf/mini_dataset_pdf/351735_page_28.pdf
Created: pdf/mini_dataset_pdf/351735_page_29.pdf
Created: pdf/mini_dataset_pdf/351735_page_30.pdf
Created: pdf/mini_dataset_pdf/351735_page_31.pdf
Created: pdf/mini_dataset_pdf/351735_page_32.pdf
Created: pdf/mini_dataset_pdf/351735_page_33.pdf
Created: pdf/mini_dataset_pdf/351735_page_34.pdf
Created: pdf/mini_dataset_pdf/351735_page_35.pdf
Created: pdf/mini_dataset_pdf/351735_page_36.pdf
Created: pdf/mini_dataset_pdf/351735_page_37.pdf
Created: pdf/mini_dataset_pdf/351735_page_38.pdf
Created: pdf/mini_dataset_pdf/351735_page_39.pdf
Created: pdf/mini_dataset_pdf/351735_page_40.pdf
Created: pdf/mini_da

In [ ]:
for pdf_file in os.listdir(miniPDFDataset):
    if 

In [18]:
for pdf_file in os.listdir(miniPDFDataset):
    if 'page' not in pdf_file:
        pdf_splitter(pdf_file)

Created: pdf/mini_dataset_pdf/209703_page_1.pdf


Now we'll do some conversions.

In [19]:
for pdf_file in os.listdir(miniPDFDataset):
    if 'page' not in pdf_file:
        os.remove(os.path.join(miniPDFDataset, pdf_file))

In [21]:
pdf_files = glob.glob(miniPDFDataset + '/*.pdf')
for pdf_file in pdf_files:
    basename = os.path.splitext(os.path.basename(pdf_file))[0]
    #print(basename)
    pngoutName = basename + '.png'
    #print(pngoutName)
    pngout = os.path.join(pngFromTop, pngoutName)
    print(pngout)
    subprocess.call(['convert', '-density', '300', pdf_file, pngout]) # 72 dpi is the default value

pdf/mini_dataset_png/134108_page_18.png
pdf/mini_dataset_png/134108_page_28.png
pdf/mini_dataset_png/236213_page_11.png
pdf/mini_dataset_png/351735_page_24.png
pdf/mini_dataset_png/351735_page_30.png
pdf/mini_dataset_png/08796_page_3.png
pdf/mini_dataset_png/35208_page_1.png
pdf/mini_dataset_png/66043_page_2.png
pdf/mini_dataset_png/134108_page_32.png
pdf/mini_dataset_png/134108_page_38.png
pdf/mini_dataset_png/105172_page_238.png
pdf/mini_dataset_png/289624_page_4.png
pdf/mini_dataset_png/351735_page_10.png
pdf/mini_dataset_png/04216_page_7.png
pdf/mini_dataset_png/390590_page_1.png
pdf/mini_dataset_png/55121_page_9.png
pdf/mini_dataset_png/105172_page_217.png
pdf/mini_dataset_png/105172_page_137.png
pdf/mini_dataset_png/221917_page_3.png
pdf/mini_dataset_png/39124_page_17.png
pdf/mini_dataset_png/105172_page_84.png
pdf/mini_dataset_png/105172_page_10.png
pdf/mini_dataset_png/407960_page_5.png
pdf/mini_dataset_png/221917_page_18.png
pdf/mini_dataset_png/105172_page_171.png
pdf/mini_da

pdf/mini_dataset_png/53481_page_1.png
pdf/mini_dataset_png/311080_page_1.png
pdf/mini_dataset_png/351735_page_39.png
pdf/mini_dataset_png/105172_page_48.png
pdf/mini_dataset_png/105172_page_141.png
pdf/mini_dataset_png/351735_page_57.png
pdf/mini_dataset_png/38776_page_14.png
pdf/mini_dataset_png/43276_page_65.png
pdf/mini_dataset_png/351735_page_20.png
pdf/mini_dataset_png/295034_page_3.png
pdf/mini_dataset_png/43519_page_9.png
pdf/mini_dataset_png/43276_page_17.png
pdf/mini_dataset_png/105172_page_188.png
pdf/mini_dataset_png/105172_page_34.png
pdf/mini_dataset_png/43276_page_33.png
pdf/mini_dataset_png/246273_page_1.png
pdf/mini_dataset_png/04216_page_5.png
pdf/mini_dataset_png/53481_page_4.png
pdf/mini_dataset_png/309804_page_5.png
pdf/mini_dataset_png/236213_page_9.png
pdf/mini_dataset_png/134108_page_13.png
pdf/mini_dataset_png/43276_page_64.png
pdf/mini_dataset_png/105172_page_110.png
pdf/mini_dataset_png/105172_page_42.png
pdf/mini_dataset_png/53481_page_13.png
pdf/mini_dataset

pdf/mini_dataset_png/105172_page_44.png
pdf/mini_dataset_png/351735_page_15.png
pdf/mini_dataset_png/134108_page_47.png
pdf/mini_dataset_png/105172_page_104.png
pdf/mini_dataset_png/134108_page_51.png
pdf/mini_dataset_png/351735_page_37.png
pdf/mini_dataset_png/105172_page_82.png
pdf/mini_dataset_png/134108_page_39.png
pdf/mini_dataset_png/105172_page_235.png
pdf/mini_dataset_png/203382_page_43.png
pdf/mini_dataset_png/351735_page_13.png
pdf/mini_dataset_png/43276_page_8.png
pdf/mini_dataset_png/29075_page_2.png
pdf/mini_dataset_png/407960_page_3.png
pdf/mini_dataset_png/43276_page_9.png
pdf/mini_dataset_png/29075_page_4.png
pdf/mini_dataset_png/04075_page_2.png
pdf/mini_dataset_png/55121_page_12.png
pdf/mini_dataset_png/111506_page_1.png
pdf/mini_dataset_png/203382_page_31.png
pdf/mini_dataset_png/105172_page_2.png
pdf/mini_dataset_png/105172_page_208.png
pdf/mini_dataset_png/57465_page_3.png
pdf/mini_dataset_png/351735_page_27.png
pdf/mini_dataset_png/351735_page_5.png
pdf/mini_datas

pdf/mini_dataset_png/100340_page_4.png
pdf/mini_dataset_png/49635_page_11.png
pdf/mini_dataset_png/50109_page_15.png
pdf/mini_dataset_png/246294_page_2.png
pdf/mini_dataset_png/203382_page_20.png
pdf/mini_dataset_png/55789_page_3.png
pdf/mini_dataset_png/55121_page_7.png
pdf/mini_dataset_png/39124_page_12.png
pdf/mini_dataset_png/100340_page_1.png
pdf/mini_dataset_png/105172_page_195.png
pdf/mini_dataset_png/38776_page_18.png
pdf/mini_dataset_png/66043_page_3.png
pdf/mini_dataset_png/43276_page_60.png
pdf/mini_dataset_png/105172_page_56.png
pdf/mini_dataset_png/105172_page_135.png
pdf/mini_dataset_png/105172_page_132.png
pdf/mini_dataset_png/134108_page_52.png
pdf/mini_dataset_png/111506_page_3.png
pdf/mini_dataset_png/105172_page_123.png
pdf/mini_dataset_png/146320_page_1.png
pdf/mini_dataset_png/105172_page_209.png
pdf/mini_dataset_png/43276_page_22.png
pdf/mini_dataset_png/221917_page_1.png
pdf/mini_dataset_png/105172_page_79.png
pdf/mini_dataset_png/55121_page_20.png
pdf/mini_datas

pdf/mini_dataset_png/50109_page_5.png
pdf/mini_dataset_png/38776_page_20.png
pdf/mini_dataset_png/236213_page_12.png
pdf/mini_dataset_png/43276_page_75.png
pdf/mini_dataset_png/377645_page_2.png
pdf/mini_dataset_png/121582_page_5.png
pdf/mini_dataset_png/338220_page_2.png
pdf/mini_dataset_png/105172_page_109.png
pdf/mini_dataset_png/351735_page_4.png
pdf/mini_dataset_png/134108_page_11.png
pdf/mini_dataset_png/146320_page_2.png
pdf/mini_dataset_png/351735_page_47.png
pdf/mini_dataset_png/105172_page_202.png
pdf/mini_dataset_png/221917_page_6.png
pdf/mini_dataset_png/105172_page_231.png
pdf/mini_dataset_png/39124_page_7.png
pdf/mini_dataset_png/65447_page_2.png
pdf/mini_dataset_png/29075_page_24.png
pdf/mini_dataset_png/43276_page_55.png
pdf/mini_dataset_png/66043_page_4.png
pdf/mini_dataset_png/145924_page_2.png
pdf/mini_dataset_png/43519_page_18.png
pdf/mini_dataset_png/55121_page_28.png
pdf/mini_dataset_png/39124_page_4.png
pdf/mini_dataset_png/55789_page_4.png


Converted!

# Make Larger Datasets

In [3]:
current_dir = '/data1/dbashir/Project/Summer2018/'
topFolder = 'pdf'
resultsFolder = 'results_top50'
intermedFolder = 'results_intermediate'
pdfSmallFolder = 'small_dataset_pdf'
pngSmallFolder = 'small_dataset_png'

dataSetSize = 700

resultsPath = os.path.join(topFolder,resultsFolder)
intermedPath = os.path.join(topFolder, intermedFolder)
intermedFromTop = os.path.join(current_dir, intermedPath)

pdfDir = os.path.join(current_dir, topFolder)
smallPDFDataset = os.path.join(topFolder, pdfSmallFolder)
smallPNGDataset = os.path.join(pdfDir, pngSmallFolder)
pngFromTop = os.path.join(topFolder, pngSmallFolder)
pdfFromTop = os.path.join(pdfDir, pdfSmallFolder)
rotPngFromTop = os.path.join(topFolder, 'small_dataset_png_rot')

fileList = []

print(pdfDir)

/data1/dbashir/Project/Summer2018/pdf


In [6]:
g = glob.glob(os.path.join(intermedPath,'*.pdf'))
shuf = np.random.permutation(g)
for i in range(dataSetSize):
    name = shuf[i]
    name = name[25:]
    #os.rename(shuf[i], os.path.join(extra_dataset, name)) # also add so we can look at just these 5
    os.rename(shuf[i], os.path.join(smallPDFDataset, name))

In [6]:
def pdf_splitter_small(path):
    fname = os.path.splitext(os.path.basename(path))[0]
    
    #print(path)
    path = os.path.join(smallPDFDataset, path)
    #print(path)
 
    pdf = PdfFileReader(open(path,"rb"))
    for page in range(pdf.getNumPages()):
        pdf_writer = PdfFileWriter()
        pdf_writer.addPage(pdf.getPage(page))
 
        output_filename = '{}_page_{}.pdf'.format(
            fname, page+1)
    
        output_filename = os.path.join(smallPDFDataset, output_filename)
 
        with open(output_filename, 'wb') as out:
            pdf_writer.write(out)
 
        print('Created: {}'.format(output_filename))

In [2]:
for pdf_file in os.listdir(smallPDFDataset):
    try:
        pdf_splitter_small(pdf_file)
    except:
        continue

NameError: name 'smallPDFDataset' is not defined

In [23]:
for pdf_file in os.listdir(smallPDFDataset):
    if 'page' not in pdf_file:
        os.remove(os.path.join(smallPDFDataset, pdf_file))

In [7]:
pdf_files = glob.glob(smallPDFDataset + '/*.pdf')
for pdf_file in pdf_files:
    basename = os.path.splitext(os.path.basename(pdf_file))[0]
    #print(basename)
    pngoutName = basename + '.png'
    #print(pngoutName)
    pngout = os.path.join(pngFromTop, pngoutName)
    print(pngout)
    if pngoutName not in os.listdir(pngFromTop):
        subprocess.call(['convert', '-density', '300', pdf_file, pngout]) # 72 dpi is the default value
    else:
        print("already converted")
        continue

pdf/small_dataset_png/279976_page_39.png
already converted
pdf/small_dataset_png/06999_page_11.png
already converted
pdf/small_dataset_png/240042_page_23.png
already converted
pdf/small_dataset_png/192497_page_2.png
already converted
pdf/small_dataset_png/00066_page_12.png
already converted
pdf/small_dataset_png/28605_page_1.png
already converted
pdf/small_dataset_png/497439_page_7.png
already converted
pdf/small_dataset_png/106406_page_112.png
already converted
pdf/small_dataset_png/106406_page_131.png
already converted
pdf/small_dataset_png/40843_page_10.png
already converted
pdf/small_dataset_png/26790_page_33.png
already converted
pdf/small_dataset_png/65145_page_18.png
already converted
pdf/small_dataset_png/297229_page_95.png
already converted
pdf/small_dataset_png/481267_page_5.png
already converted
pdf/small_dataset_png/26655_page_68.png
already converted
pdf/small_dataset_png/28896_page_8.png
already converted
pdf/small_dataset_png/427935_page_1.png
already converted
pdf/small

already converted
pdf/small_dataset_png/26655_page_105.png
already converted
pdf/small_dataset_png/268479_page_3.png
already converted
pdf/small_dataset_png/30415_page_115.png
already converted
pdf/small_dataset_png/51782_page_29.png
already converted
pdf/small_dataset_png/40843_page_20.png
already converted
pdf/small_dataset_png/311740_page_3.png
already converted
pdf/small_dataset_png/400052_page_4.png
already converted
pdf/small_dataset_png/344911_page_3.png
already converted
pdf/small_dataset_png/35997_page_6.png
already converted
pdf/small_dataset_png/30415_page_81.png
already converted
pdf/small_dataset_png/222198_page_3.png
already converted
pdf/small_dataset_png/106406_page_28.png
already converted
pdf/small_dataset_png/445835_page_46.png
already converted
pdf/small_dataset_png/106406_page_218.png
already converted
pdf/small_dataset_png/247511_page_4.png
already converted
pdf/small_dataset_png/65144_page_42.png
already converted
pdf/small_dataset_png/46592_page_1.png
already co

already converted
pdf/small_dataset_png/435137_page_17.png
already converted
pdf/small_dataset_png/51782_page_15.png
already converted
pdf/small_dataset_png/06999_page_2.png
already converted
pdf/small_dataset_png/26015_page_14.png
already converted
pdf/small_dataset_png/02231_page_4.png
already converted
pdf/small_dataset_png/241697_page_11.png
already converted
pdf/small_dataset_png/240042_page_52.png
already converted
pdf/small_dataset_png/38410_page_9.png
already converted
pdf/small_dataset_png/445712_page_72.png
already converted
pdf/small_dataset_png/114536_page_56.png
already converted
pdf/small_dataset_png/06357_page_6.png
already converted
pdf/small_dataset_png/442916_page_10.png
already converted
pdf/small_dataset_png/13598_page_4.png
already converted
pdf/small_dataset_png/12980_page_46.png
already converted
pdf/small_dataset_png/107236_page_58.png
already converted
pdf/small_dataset_png/293550_page_1.png
already converted
pdf/small_dataset_png/261900_page_18.png
already con

already converted
pdf/small_dataset_png/348450_page_12.png
already converted
pdf/small_dataset_png/106406_page_99.png
already converted
pdf/small_dataset_png/330059_page_14.png
already converted
pdf/small_dataset_png/33056_page_8.png
already converted
pdf/small_dataset_png/178105_page_12.png
already converted
pdf/small_dataset_png/00100_page_19.png
already converted
pdf/small_dataset_png/355461_page_7.png
already converted
pdf/small_dataset_png/433909_page_3.png
already converted
pdf/small_dataset_png/42119_page_16.png
already converted
pdf/small_dataset_png/261900_page_44.png
already converted
pdf/small_dataset_png/19386_page_43.png
already converted
pdf/small_dataset_png/426281_page_12.png
already converted
pdf/small_dataset_png/89141_page_111.png
already converted
pdf/small_dataset_png/02028_page_121.png
already converted
pdf/small_dataset_png/297229_page_50.png
already converted
pdf/small_dataset_png/19386_page_129.png
already converted
pdf/small_dataset_png/114536_page_9.png
alrea

pdf/small_dataset_png/02028_page_29.png
pdf/small_dataset_png/347782_page_15.png
pdf/small_dataset_png/44700_page_55.png
pdf/small_dataset_png/51033_page_10.png
pdf/small_dataset_png/107236_page_15.png
pdf/small_dataset_png/38410_page_50.png
pdf/small_dataset_png/25425_page_8.png
pdf/small_dataset_png/297229_page_24.png
pdf/small_dataset_png/04372_page_3.png
pdf/small_dataset_png/26994_page_1.png
pdf/small_dataset_png/348450_page_5.png
pdf/small_dataset_png/65218_page_17.png
pdf/small_dataset_png/27331_page_130.png
pdf/small_dataset_png/104004_page_4.png
pdf/small_dataset_png/01225_page_7.png
pdf/small_dataset_png/445712_page_7.png
pdf/small_dataset_png/61169_page_3.png
pdf/small_dataset_png/297229_page_191.png
pdf/small_dataset_png/454022_page_1.png
pdf/small_dataset_png/64778_page_3.png
pdf/small_dataset_png/445835_page_62.png
pdf/small_dataset_png/106406_page_159.png
pdf/small_dataset_png/01927_page_6.png
pdf/small_dataset_png/36038_page_4.png
pdf/small_dataset_png/106406_page_195.p

...

pdf/small_dataset_png/12980_page_37.png
pdf/small_dataset_png/26655_page_6.png
pdf/small_dataset_png/395118_page_15.png
pdf/small_dataset_png/28896_page_16.png
pdf/small_dataset_png/44698_page_88.png
pdf/small_dataset_png/19386_page_64.png
pdf/small_dataset_png/19386_page_77.png
pdf/small_dataset_png/228493_page_108.png
pdf/small_dataset_png/25381_page_11.png
pdf/small_dataset_png/228493_page_177.png
pdf/small_dataset_png/134695_page_1.png
pdf/small_dataset_png/279976_page_7.png
pdf/small_dataset_png/106406_page_22.png
pdf/small_dataset_png/297229_page_132.png
pdf/small_dataset_png/44698_page_91.png
pdf/small_dataset_png/112431_page_23.png
pdf/small_dataset_png/30550_page_22.png
pdf/small_dataset_png/297229_page_85.png
pdf/small_dataset_png/44698_page_49.png
pdf/small_dataset_png/455910_page_32.png
pdf/small_dataset_png/512624_page_3.png
pdf/small_dataset_png/30521_page_102.png
pdf/small_dataset_png/03205_page_4.png
pdf/small_dataset_png/25425_page_29.png
pdf/small_dataset_png/246957_p

pdf/small_dataset_png/35620_page_8.png
pdf/small_dataset_png/324076_page_4.png
pdf/small_dataset_png/240042_page_10.png
pdf/small_dataset_png/27331_page_66.png
pdf/small_dataset_png/25156_page_2.png
pdf/small_dataset_png/20805_page_6.png
pdf/small_dataset_png/01507_page_22.png
pdf/small_dataset_png/20997_page_69.png
pdf/small_dataset_png/298137_page_2.png
pdf/small_dataset_png/445712_page_90.png
pdf/small_dataset_png/111610_page_159.png
pdf/small_dataset_png/241875_page_2.png
pdf/small_dataset_png/44700_page_41.png
pdf/small_dataset_png/495995_page_10.png
pdf/small_dataset_png/65218_page_48.png
pdf/small_dataset_png/54069_page_42.png
pdf/small_dataset_png/55136_page_23.png
pdf/small_dataset_png/253953_page_2.png
pdf/small_dataset_png/35565_page_2.png
pdf/small_dataset_png/89141_page_142.png
pdf/small_dataset_png/04565_page_2.png
pdf/small_dataset_png/111610_page_218.png
pdf/small_dataset_png/60196_page_1.png
pdf/small_dataset_png/106406_page_221.png
pdf/small_dataset_png/01246_page_11.

pdf/small_dataset_png/43372_page_22.png
pdf/small_dataset_png/304048_page_43.png
pdf/small_dataset_png/00853_page_35.png
pdf/small_dataset_png/111610_page_83.png
pdf/small_dataset_png/274794_page_4.png
pdf/small_dataset_png/166577_page_1.png
pdf/small_dataset_png/29067_page_5.png
pdf/small_dataset_png/444134_page_2.png
pdf/small_dataset_png/65144_page_37.png
pdf/small_dataset_png/347782_page_48.png
pdf/small_dataset_png/112719_page_7.png
pdf/small_dataset_png/30521_page_18.png
pdf/small_dataset_png/440301_page_12.png
pdf/small_dataset_png/397785_page_7.png
pdf/small_dataset_png/497326_page_10.png
pdf/small_dataset_png/114536_page_23.png
pdf/small_dataset_png/28896_page_2.png
pdf/small_dataset_png/44700_page_155.png
pdf/small_dataset_png/65144_page_2.png
pdf/small_dataset_png/228493_page_113.png
pdf/small_dataset_png/106406_page_3.png
pdf/small_dataset_png/107236_page_96.png
pdf/small_dataset_png/330059_page_18.png
pdf/small_dataset_png/51179_page_1.png
pdf/small_dataset_png/111610_page

pdf/small_dataset_png/380499_page_21.png
pdf/small_dataset_png/192497_page_3.png
pdf/small_dataset_png/44698_page_39.png
pdf/small_dataset_png/297229_page_269.png
pdf/small_dataset_png/00706_page_2.png
pdf/small_dataset_png/43255_page_7.png
pdf/small_dataset_png/26905_page_4.png
pdf/small_dataset_png/03354_page_16.png
pdf/small_dataset_png/54069_page_16.png
pdf/small_dataset_png/02194_page_5.png
pdf/small_dataset_png/41059_page_6.png
pdf/small_dataset_png/30550_page_45.png
pdf/small_dataset_png/27331_page_20.png
pdf/small_dataset_png/42632_page_5.png
pdf/small_dataset_png/43372_page_78.png
pdf/small_dataset_png/09431_page_7.png
pdf/small_dataset_png/02028_page_126.png
pdf/small_dataset_png/26535_page_1.png
pdf/small_dataset_png/25156_page_28.png
pdf/small_dataset_png/19386_page_10.png
pdf/small_dataset_png/435137_page_10.png
pdf/small_dataset_png/53432_page_1.png
pdf/small_dataset_png/08736_page_9.png
pdf/small_dataset_png/48684_page_2.png
pdf/small_dataset_png/26655_page_104.png
pdf/s

KeyboardInterrupt: 

# Medium Dataset

Note: I couldn't get out all of the handwritten scores it seems. Other things to consider might be removing anything where publisher date says it's before 1800 or smth, things like that.

In [ ]:
current_dir = '/data1/dbashir/Project/Summer2018/'
topFolder = 'pdf'
resultsFolder = 'results_top50'
intermedFolder = 'results_intermediate'
pdfMedFolder = 'medium_dataset_pdf'
pngMedFolder = 'medium_dataset_png'

dataSetSize = 7000

resultsPath = os.path.join(topFolder,resultsFolder)
intermedPath = os.path.join(topFolder, intermedFolder)
intermedFromTop = os.path.join(current_dir, intermedPath)

pdfDir = os.path.join(current_dir, topFolder)
medPDFDataset = os.path.join(topFolder, pdfMedFolder)
medPNGDataset = os.path.join(pdfDir, pngMedFolder)
pngFromTop = os.path.join(topFolder, pngMedFolder)
pdfFromTop = os.path.join(pdfDir, pdfMedFolder)
rotPngFromTop = os.path.join(topFolder, 'medium_dataset_png_rot')

fileList = []

print(pdfDir)

In [ ]:
g = glob.glob(os.path.join(intermedPath,'*.pdf'))
shuf = np.random.permutation(g)
for i in range(dataSetSize):
    name = shuf[i]
    name = name[25:]
    #os.rename(shuf[i], os.path.join(extra_dataset, name)) # also add so we can look at just these 5
    os.rename(shuf[i], os.path.join(medPDFDataset, name))

In [ ]:
def pdf_splitter_med(path):
    fname = os.path.splitext(os.path.basename(path))[0]
    
    #print(path)
    path = os.path.join(medPDFDataset, path)
    #print(path)
 
    pdf = PdfFileReader(open(path,"rb"))
    for page in range(pdf.getNumPages()):
        pdf_writer = PdfFileWriter()
        pdf_writer.addPage(pdf.getPage(page))
 
        output_filename = '{}_page_{}.pdf'.format(
            fname, page+1)
    
        output_filename = os.path.join(medPDFDataset, output_filename)
 
        with open(output_filename, 'wb') as out:
            pdf_writer.write(out)
 
        print('Created: {}'.format(output_filename))

In [ ]:
for pdf_file in os.listdir(medPDFDataset):
    try:
        pdf_splitter_med(pdf_file)
    except:
        continue

In [ ]:
for pdf_file in os.listdir(medPDFDataset):
    if 'page' not in pdf_file:
        os.remove(os.path.join(medPDFDataset, pdf_file))

In [ ]:
pdf_files = glob.glob(medPDFDataset + '/*.pdf')
for pdf_file in pdf_files:
    basename = os.path.splitext(os.path.basename(pdf_file))[0]
    #print(basename)
    pngoutName = basename + '.png'
    #print(pngoutName)
    pngout = os.path.join(pngFromTop, pngoutName)
    print(pngout)  
    if pngoutName not in os.listdir(pngFromTop):
        subprocess.call(['convert', '-density', '300', pdf_file, pngout]) # 72 dpi is the default value
    else:
        print("already converted")
        continue

# Large Dataset

TODO: we need to make sure we correctly crawl the entire database here (not just top 50 composers) since we want this dataset to have a size of 70000. We could also do something like what we did before w the results_intermediate:
- crawl the full dataset
- for i in range 70000: move things to results_intermediate which will actually be final
- convert everything to pages
- convert all to PNG

This seems like it'll take forever though, given what we've seen so far.

In [ ]:
current_dir = '/data1/dbashir/Project/Summer2018'
topFolder = 'pdf'
toResults = '/data1/dbashir/Project/'
resultsFolder = 'score_scrape/results/'
intermedFolder = 'results_intermediate_large'
pdfLargeFolder = 'large_dataset_pdf'
pngLargeFolder = 'large_dataset_png'

dataSetSize = 70000

resultsPath = os.path.join(toResults,resultsFolder)
intermedPath = os.path.join(topFolder, intermedFolder)
intermedFromTop = os.path.join(current_dir, intermedPath)

pdfDir = os.path.join(current_dir, topFolder)
largePDFDataset = os.path.join(topFolder, pdfLargeFolder)
largePNGDataset = os.path.join(pdfDir, pngLargeFolder)
pngFromTop = os.path.join(topFolder, pngLargeFolder)
pdfFromTop = os.path.join(pdfDir, pdfLargeFolder)
rotPngFromTop = os.path.join(topFolder, 'large_dataset_png_rot')

fileList = []

print(pdfDir)

In [ ]:
def make_bad_pdfs(html_path):
    bad_names = []
    with open(html_path) as f:
        soup = bsoup(f, "html.parser")

    for div in soup.find_all('div', class_ = 'we'):
        # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
        trs = div.find_all('tr') # also get no publisher
        for tr in trs:
            tr = str(tr)
            try:
                if 'Pub' not in tr:
                    value_on = tr[tr.find('indexes='):]
                    bad_name = value_on[8:14]
                    if bad_name[-1] == '/':
                        # if it's a slash, there's something else after it
                        bad_name_two = value_on[14:20]
                        # check if last thing is actually a digit, in case of 5-digit ID
                        if not bad_name_two[-1].isdigit():
                            bad_name_two = bad_name_two[:-1]
                        bad_pdf_two = bad_name_two + '.pdf'
                        bad_names.append(bad_pdf_two)
                    if not bad_name[-1].isdigit():
                        # make could be a 5-digit ID
                        bad_name = bad_name[:-1]
                    bad_pdf = bad_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
                    bad_names.append(bad_pdf) # append this to a list of bad name
            except:
                continue
        div = str(div)
        try:
            if 'Manuscript' in div or 'manuscript' in div:
                value_on = div[div.find('indexes='):] # this is where the ID should show up on a page
                bad_name = value_on[8:14] # just get the id, assuming 6 digits
                if bad_name[-1] == '/':
                    # if it's a slash, there's something else after it
                    bad_name_two = value_on[14:20]
                    # check if last thing is actually a digit, in case of 5-digit ID
                    if not bad_name_two[-1].isdigit():
                        bad_name_two = bad_name_two[:-1]
                    bad_pdf_two = bad_name_two + '.pdf'
                    bad_names.append(bad_pdf_two)
                if not bad_name[-1].isdigit():
                    # make could be a 5-digit ID
                    bad_name = bad_name[:-1]
                bad_pdf = bad_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
                bad_names.append(bad_pdf) # append this to a list of bad names
        except:
            continue
            
    return bad_names

In [ ]:
def make_score_pdfs(html_path):
    score_names = []
    divList = []
    with open(html_path) as f:
        soup = bsoup(f, "html.parser")
    for div in soup.find_all('div', class_ = 'we'):
        # we will contain both an ID and the publisher info for a piece so we can make sure it's not handwritten
        div = str(div)
        if 'Score' in div or 'score' in div:
            divList.append(div)
    for div in divList:
        try:
            value_on = div[div.find('indexes='):]
            score_name = value_on[8:14]
            if score_name[-1] == '/':
                    # if it's a slash, there's something else after it
                    score_name_two = value_on[14:20]
                    # check if last thing is actually a digit, in case of 5-digit ID
                    if not score_name_two[-1].isdigit():
                        score_name_two = score_name_two[:-1]
                    score_pdf_two = score_name_two + '.pdf'
                    score_names.append(score_pdf_two)
            if not score_name[-1].isdigit():
                # make could be a 5-digit ID
                score_name = score_name[:-1]
            score_pdf = score_name + '.pdf' # add to .pdf so we have the actual filename we want to avoid/delete if it's in our thing
            score_names.append(score_pdf) # append this to a list of bad names
        except:
            continue
        
    return score_names

In [ ]:
#using os.walk, move all the .pdf files to an intermediate folder for EZ access
all_bad_names = []
all_score_names = []
for subdir, dirs, files in os.walk(resultsPath):
    for pdf_file in files:
        if 'pdf' in pdf_file:
            #move it to intermediate file for shuffling
            html_path = os.path.join(subdir, 'html.txt'j)
            bad_names = make_bad_pdfs(html_path)
            score_names = make_score_pdfs(html_path)
            for bad_name in bad_names:
                all_bad_names.append(bad_name)
            for score_name in score_names:
                all_score_names.append(score_name)
            if pdf_file in bad_names or pdf_file in score_names:
                print("skipping file: " + str(pdf_file))
                continue # skip if it's a handwritten or orchestral score
            if pdf_file not in all_bad_names or pdf_file not in all_score_names:
                pdf_file_path = os.path.join(subdir, pdf_file)
                final_path = os.path.join(intermedPath, pdf_file)
                shutil.copy(pdf_file_path, final_path)